In [1]:
# import module list is independent on respective file.
import yfinance as yf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

from datetime import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,  MinMaxScaler
from scipy.stats import mstats
import japanize_matplotlib
from tqdm.notebook import tqdm



In [2]:
# load correct trading day format

csv_file_path = './data/Train_data.csv' 
Train_data = pd.read_csv(csv_file_path, index_col=0, thousands=",", parse_dates=True) 
# read different lenghth for cutting
skip_to_date = "2005-10-01" 


# input the ticker
ticker = "IYR" #Real Estate SPDR  



#     Load saved share price file
#file_path = '~/data/' #ダウンロードした人用
file_path = './data/' #自分用
file_name = ticker
file_extension = '.csv'
price_list = pd.read_csv(file_path + file_name + file_extension,thousands=",", index_col=0, parse_dates=True)           

In [4]:
# use correct format's index to retrieve the trading data
post_fundamental = Train_data[skip_to_date:]  

idx = np.searchsorted(price_list.index, post_fundamental.index)
#documents = _list.iloc[idx]
fold_index= 3

In [5]:
def folding_train_data(df, index2):
    try:
        if "index" in df.columns.to_list():
            name = "index"
        else:
            df = df.reset_index(drop=False)
            name = "Date"
        df = df[df.index % index2==0]
        
        
        
    except Exception as e:
        raise e
    else:
        
        df.set_index(name,drop=True, inplace=True)
    
    return df 




In [6]:
fold_index= 3
#post_fundamental.rename_axis('Date', inplace=True)
post_fundamental = post_fundamental.loc[skip_to_date:, :].asfreq(str(fold_index)+'M')

In [7]:
post_fundamental

,Monthly Open,futures,Outlays - Receipts,US Bond,Outlays,Receipts,Government Account Series,State and Local Government Series,Treasury Inflation-Protected Securities,Principal Amount Balance,...,Bond(Net Sales Amount),Saving Bond typeI,Saving Bond typeEE,Principal Amount Increase,Net Unamortized Amount Increase,Principal Amount Decrease,Net Unamortized Amount Decrease,Accrued Interest Payable Amount,Accrued Interest Increase,Accrued Interest decrease
Date,,,,,,,,,,,,,,,,,,,,,
2005-10-31,2005/10/1,108.45,47277.0,4.559,196764.0,149488.0,3426716.0,226929.0,324026.0,4650516.0,...,NaN,1.436727e+08,2471203.68,332366.0,-3317.0,283089.0,-2854.0,42249.0,14724.0,7436.0
2006-01-31,2006/1/1,108.44,-20964.0,4.527,209045.0,230010.0,3523210.0,233441.0,346861.0,4724637.0,...,NaN,3.215712e+08,4665751.01,347391.0,-3440.0,337575.0,-3407.0,42944.0,10920.0,8172.0
2006-04-30,2006/4/1,105.58,-118841.0,5.069,196249.0,315090.0,3589106.0,238663.0,367656.0,4819949.0,...,NaN,9.420792e+07,2260916.67,296608.0,-3287.0,349144.0,-3966.0,45576.0,14400.0,5445.0
2006-07-31,2006/7/1,106.03,33164.0,4.988,192925.0,159761.0,3683171.0,237425.0,393657.0,4819732.0,...,NaN,1.446631e+07,2334342.28,330464.0,-4380.0,307936.0,-4307.0,45001.0,16296.0,12081.0
2006-10-31,2006/10/1,108.22,49321.0,4.606,217014.0,167693.0,3762745.0,241728.0,415359.0,4880611.0,...,NaN,1.490999e+07,2494720.03,346502.0,-3661.0,309012.0,-4072.0,45743.0,15541.0,10917.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-31,2022/7/1,121.14,211052.0,2.642,480383.0,269331.0,6944610.0,103866.0,1789336.0,23952579.0,...,1421300.0,2.342998e+09,8918548.59,1079881.0,-8236.0,1039462.0,-4215.0,102009.0,49136.0,23537.0
2022-10-31,2022/10/1,110.88,87874.0,4.077,406374.0,318500.0,7188183.0,104454.0,1860886.0,24352542.0,...,14469700.0,6.781886e+09,17660717.08,1330585.0,-19138.0,1277236.0,-8788.0,94100.0,31762.0,18845.0
2023-01-31,2023/1/1,115.08,38784.0,3.529,486072.0,447288.0,7024099.0,99115.0,1870818.0,24583994.0,...,46779200.0,4.223309e+09,28361273.15,1774182.0,-13074.0,1707781.0,-13278.0,101418.0,33289.0,31384.0


In [7]:
DS = post_fundamental.loc[:,'Outlays - Receipts'].values

US_Bond = post_fundamental.loc[:,'US Bond'].values
Futures = post_fundamental.loc[:,'futures'].values

Receipts = post_fundamental.loc[:,'Receipts'].values

Outlays = post_fundamental.loc[:,'Outlays'].values

SLGS = post_fundamental.loc[:,'State and Local Government Series'].values

GAS = post_fundamental.loc[:,'Government Account Series'].values
TIPS = post_fundamental.loc[:,'Treasury Inflation-Protected Securities'].values

PAB = post_fundamental.loc[:,'Principal Amount  Balance'].values
AIPA = post_fundamental.loc[:,'Accrued Interest Payable Amount'].values

NUAB = post_fundamental.loc[:,'Net Unamortized Amount Balance'].values

Bill = post_fundamental.loc[:,'Bill(Net Sales Amount)'].values
Note = post_fundamental.loc[:,'Note(Net Sales Amount)'].values
Bond = post_fundamental.loc[:,'Bond(Net Sales Amount)'].values



SBI = post_fundamental.loc[:,'Saving Bond typeI'].values
SBEE = post_fundamental.loc[:,'Saving Bond typeEE'].values

PAI = post_fundamental.loc[:,'Principal Amount Increase'].values
AII = post_fundamental.loc[:,'Accrued Interest Increase'].values
NUAI = post_fundamental.loc[:,'Net Unamortized Amount Increase'].values

PAD = post_fundamental.loc[:,'Principal Amount Decrease'].values
AID = post_fundamental.loc[:,'Accrued Interest decrease'].values
NUAD = post_fundamental.loc[:,'Net Unamortized Amount Decrease'].values



OK


In [30]:
file_name

'IYR'

In [8]:
# Goal is to anticipate the sign of future rate change from the financial data of the current monthly.
# If the future changes is + , we assign 1, otherwise 0,  to Future change value of the current quarter


# Returned Sales Amount.

# predict the FFO direction

data_2 = []
data_2 = pd.DataFrame(data_2)

# 2023 09 DS, Receipts, GAS, NUA
# 2023 09 DS Receipts Bond NUAB
data_2['DS'] = DS

data_2['SLGS'] = SLGS
data_2["GAS"] = GAS
data_2['Receipts'] = Receipts

# Principal Amount Balance
data_2['PAB'] = PAB
data_2['AIPA'] = AIPA


data_2['Bill'] = Bill
data_2['Note'] = Note
data_2['Bond'] = Bond
'''
data_2['Bill2'] = Bill2
data_2['Note2'] = Note2
data_2['Bond2'] = Bond2
'''
data_2['NUAB'] = NUAB
data_2['SBI'] = SBI
data_2['SBEE'] = SBEE
data_2['AII'] = AII
data_2['PAI'] = PAI
data_2['NUAI'] = NUAI

data_2['PAD'] = PAD 
data_2['AID'] = AID
data_2['NUAD'] = NUAD

In [13]:
data_2.corr()

,DS,SLGS,GAS,Receipts,PAB,AIPA,Bill,Note,Bond,NUAB
DS,1.000000,-0.087813,0.136473,-0.583445,0.160040,0.143155,0.000685,-0.100779,0.033773,0.056007
SLGS,-0.087813,1.000000,-0.804083,-0.355778,-0.773119,-0.769848,-0.276140,-0.393512,0.032726,0.184365
GAS,0.136473,-0.804083,1.000000,0.511731,0.977838,0.949596,0.569580,0.530074,0.204744,-0.451222
Receipts,-0.583445,-0.355778,0.511731,1.000000,0.532307,0.507262,0.329219,0.283691,0.057288,-0.265834
PAB,0.160040,-0.773119,0.977838,0.532307,1.000000,0.959069,0.575292,0.493855,0.203522,-0.441157
AIPA,0.143155,-0.769848,0.949596,0.507262,0.959069,1.000000,0.664232,0.445267,0.143203,-0.541535
Bill,0.000685,-0.276140,0.569580,0.329219,0.575292,0.664232,1.000000,0.649021,0.434754,-0.928839
Note,-0.100779,-0.393512,0.530074,0.283691,0.493855,0.445267,0.649021,1.000000,0.663422,-0.608383
Bond,0.033773,0.032726,0.204744,0.057288,0.203522,0.143203,0.434754,0.663422,1.000000,-0.419914
NUAB,0.056007,0.184365,-0.451222,-0.265834,-0.441157,-0.541535,-0.928839,-0.608383,-0.419914,1.000000


In [10]:



    


train_len = 7


#namelist = [['Receipts', 'AID', 'NUAI', 'AIPA']]
#namelist = [['PAD', 'NUAI', 'NUAD', 'NUAB']]
namelist = ['NUAB', 'DS', 'Receipts', 'PAB'] 

df = data_2[namelist]

# 欠損値を列の1つ手前の値で埋める
df = df.dropna(how='any').dropna(how='all', axis=0)
df = df.fillna(method='ffill')
X = df.values

USBond_change = np.where(US_Bond[1:] > US_Bond[0:-1], 1, 0)

USBond_change = np.append(USBond_change,1)   # adjustment, not being used


y = USBond_change


y_test = []
rate_prediction = []

result = []
result = pd.DataFrame(result)
for i in range(len(y)-train_len+1):
    X_train = X[0:train_len+i-1,:]    # train until previous period, cuz y is forecasting next period
    y_train = y[0:train_len+i-1]
    X_predict = X[0:train_len+i,:] # use most recent reporting period for forecasting

    # Winsorize top 1% and bottom 1% of points.
    # Apply on X_train and X_test separately
    X_train = mstats.winsorize(X_train, limits = [0.01, 0.01])
    X_predict = mstats.winsorize(X_predict, limits = [0.01, 0.01])

    sc = StandardScaler()
    # Fit to training data and then transform it
    X_train = sc.fit_transform(X_train) # SVM is unsuitable to raw larger-scale number, However, Standard saves this.
    X_predict = sc.transform(X_predict) 

    y_test.append(y[train_len+i-1]) # 6, 7, 8, 9, 10, 11.......

    # Initialize svm, rbf is a default kernel
    classifier_rbf = SVC(C=45, kernel = 'rbf', gamma = 'auto', random_state=0)


    # Fit the model on training data

    # Make a prediction on testing data
    try:
        classifier_rbf.fit(X_train, y_train)
        y_pred_rbf = classifier_rbf.predict(X_predict[train_len -1+i,:].reshape(1,-1))
    except:
        pass


    rate_prediction = np.append(rate_prediction,y_pred_rbf)





result['predict'] = rate_prediction


from sklearn.metrics import matthews_corrcoef
value= matthews_corrcoef(y_test, rate_prediction)

print('corr:', value)


result.index = [i+timedelta(days=1) for i in post_fundamental.index[train_len-1:]]






corr: 0.15498260496951669


In [ ]:
1126 

〇corr: 0.15498260496951669 Index(['NUAB', 'DS', 'Receipts', 'PAB'], dtype='object') 45
    
    ['NUAD', 'AII', 'PAD', 'NUAI']


In [41]:
# Export data to CSV file
export = True
file_path = './prediction/'
file_name = "IYR3mo"
file_extension = '.csv'

#     new_copy.to_csv(file_path + file_name + file_extension)
if export:
    data = result
    data.to_csv(file_path + file_name + file_extension)